In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Parcial 2 - María Alejandra Velásquez (202113333)- Juan Pablo Camacho (202110977)

 # ======================================
# BLOQUE 1.5: EDA Reforzado (para Reporte)
# ======================================
import pandas as pd
import zipfile
import io

# Columnas estándar del dataset Adult
columns = [
    "age","workclass","fnlwgt","education","education_num","marital_status","occupation",
    "relationship","race","sex","capital_gain","capital_loss","hours_per_week","native_country","income"
]

# 1. Ruta del archivo que subiste a Colab
# Assuming zip_path is defined elsewhere in the notebook and points to the zip file
# zip_path = "path/to/your/adult.zip" # Define zip_path if not already defined
# Paso 1: Subir el archivo desde tu PC
from google.colab import files
uploaded = files.upload()   # Aquí seleccionas "adult.zip"

import zipfile, os

# Paso 2: Descomprimir el zip en una carpeta
if uploaded: # Check if files were uploaded
    zip_name = list(uploaded.keys())[0]   # Obtiene el nombre real del archivo subido
    with zipfile.ZipFile(zip_name, 'r') as z:
        z.extractall("adult_data")

    # Paso 3: Revisar que se extrajo bien
    print("Archivos extraídos:", os.listdir("adult_data"))

    # Paso 4: Cargar datasets
    import pandas as pd

    columns = [
        "age","workclass","fnlwgt","education","education_num","marital_status",
        "occupation","relationship","race","sex","capital_gain","capital_loss",
        "hours_per_week","native_country","income"
    ]

    train = pd.read_csv("adult_data/adult.data", header=None, names=columns, na_values=" ?", skipinitialspace=True)
    test  = pd.read_csv("adult_data/adult.test", header=None, names=columns, na_values=" ?", skipinitialspace=True, skiprows=1)

    print("Train shape:", train.shape)
    print("Test shape:", test.shape)
    print("Valores únicos en income:", train["income"].unique(), test["income"].unique())



    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats
    from sklearn.metrics import mutual_info_score
    import warnings
    warnings.filterwarnings("ignore")
    plt.rcParams["figure.figsize"] = (8,5)


    # Limpieza mínima y etiquetas consistentes
    train["income"] = train["income"].str.strip()
    test["income"]  = test["income"].str.strip().str.strip(".")
    train = train.dropna()
    test  = test.dropna()

    # =========================
    # 1) Info general y balance de clases
    # =========================
    def describe_basic(df, name="DF"):
        print(f"=== {name} ===")
        display(df.head(3))
        print("Shape:", df.shape)
        print(df.dtypes)
        print("Valores únicos por columna (top 5):")
        for c in df.columns:
            u = df[c].dropna().unique()
            print(f" - {c}: {len(u)} {'-> ' + ', '.join(map(str,u[:5])) + ('...' if len(u)>5 else '')}")
        print("\n")

    describe_basic(train, "TRAIN")
    describe_basic(test, "TEST")

    def class_balance(df, name="DF"):
        vc = df["income"].value_counts(normalize=True).sort_index()
        print(f"Balance de clases en {name}:")
        print((vc*100).round(2).astype(str) + "%")
        ax = sns.countplot(x="income", data=df, order=sorted(df["income"].unique()))
        ax.bar_label(ax.containers[0])
        plt.title(f"Distribución de income en {name}")
        plt.show()

    class_balance(train, "TRAIN")
    class_balance(test,  "TEST")

    # =========================
    # 2) Valores faltantes y categorías raras
    # =========================
    def missing_table(df, name="DF"):
        miss = df.isna().mean().sort_values(ascending=False).to_frame("missing_rate")
        print(f"Porcentaje de NA por columna en {name}:")
        display((miss*100).round(2))
    missing_table(train, "TRAIN")
    missing_table(test,  "TEST")

    def rare_categories(df, threshold=0.01):
        print(f"Categorías raras (freq <{threshold*100:.1f}%) por columna categórica")
        cat_cols = df.select_dtypes("object").columns.drop("income")
        for c in cat_cols:
            freq = df[c].value_counts(normalize=True)
            rares = freq[freq < threshold]
            if len(rares):
                print(f" - {c}: {len(rares)} raras -> {', '.join(rares.index[:8])}{'...' if len(rares)>8 else ''}")
    rare_categories(train)

    # =========================
    # 3) Drift Train–Test (PSI simple para columnas numéricas)
    # =========================
    def psi(expected, actual, bins=10):
        # Population Stability Index (aprox), útil para comparar distribuciones
        quantiles = np.linspace(0, 1, bins+1)
        cut = np.unique(np.quantile(expected, quantiles))
        e_hist, _ = np.histogram(expected, bins=cut)
        a_hist, _ = np.histogram(actual,   bins=cut)
        e_perc = np.where(e_hist==0, 1e-6, e_hist / e_hist.sum())
        a_perc = np.where(a_hist==0, 1e-6, a_hist / a_hist.sum())
        return np.sum((a_perc - e_perc) * np.log(a_perc / e_perc))

    num_cols = train.select_dtypes(include=[np.number]).columns
    psi_vals = {}
    for c in num_cols:
        psi_vals[c] = psi(train[c].values, test[c].values, bins=10)
    psi_df = pd.Series(psi_vals, name="PSI").sort_values(ascending=False).to_frame()
    print("PSI (Train vs Test) por variable numérica (≈0 estable, >0.1 alerta, >0.25 severo):")
    display(psi_df.round(3))

    # =========================
    # 4) Distribuciones numéricas por clase (diagnóstico de separabilidad)
    # =========================
    target_map = {"<=50K":0, ">50K":1}
    train_t = train.assign(y=train["income"].map(target_map))

    for c in ["age","education_num","hours_per_week","capital_gain","capital_loss","fnlwgt"]:
        fig, ax = plt.subplots()
        sns.kdeplot(data=train, x=c, hue="income", common_norm=False, fill=True, alpha=0.4, ax=ax)
        plt.title(f"Distribución de {c} por income")
        plt.show()

    # =========================
    # 5) Relación numéricas–objetivo (correlación punto-biserial)
    # =========================
    def point_biserial_corr_num(df, target="income"):
        df_ = df.copy()
        df_["y"] = df_[target].map(target_map)
        corrs = {}
        for c in df_.select_dtypes(include=[np.number]).columns:
            if c == "y":
                continue
            r, p = stats.pointbiserialr(df_[c], df_["y"])
            corrs[c] = (r, p)
        out = pd.DataFrame(corrs, index=["r_pb","p_value"]).T.sort_values("r_pb", ascending=False)
        return out

    pbs = point_biserial_corr_num(train)
    print("Correlación punto-biserial numéricas vs income (train):")
    display(pbs.round(4))

    # =========================
    # 6) Relación categóricas–objetivo (tasa de >50K y mutual information)
    # =========================
    cat_cols = train.select_dtypes(include=["object"]).columns.drop("income")

    def rate_over_50k_by_cat(df, col):
        tab = (df.groupby(col)["income"]
               .apply(lambda s: (s==">50K").mean())
               .sort_values(ascending=False)
               .rename("rate_>50K"))
        return tab

    for c in ["workclass","education","marital_status","occupation","relationship","race","sex","native_country"]:
        tab = rate_over_50k_by_cat(train, c)
        print(f"\nTasa >50K por {c}:")
        display((tab*100).round(1).to_frame("%"))

        # Barplot top categorías (si son muchas, mostramos top 12)
        top = tab.head(min(12, len(tab)))
        ax = top.mul(100).plot(kind="bar")
        plt.ylabel("% >50K")
        plt.title(f"% >50K por {c} (Top)")
        plt.xticks(rotation=45, ha="right")
        plt.show()

    # Mutual information (categóricas vs objetivo)
    def mutual_info_categoricals(df, target="income"):
        y = df[target].map(target_map)
        mi = {}
        for c in df.select_dtypes("object").columns:
            if c == target:
                continue
            mi[c] = mutual_info_score(df[c].astype(str), y)
        return pd.Series(mi, name="MI").sort_values(ascending=False)

    mi_cat = mutual_info_categoricals(train)
    print("Mutual Information (categóricas vs income):")
    display(mi_cat.round(4).to_frame())

    # =========================
    # 7) Outliers (IQR) para variables clave
    # =========================
    def iqr_outlier_share(x):
        Q1, Q3 = np.percentile(x, [25,75])
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
        return ((x<lower) | (x>upper)).mean()

    outlier_rates = {c: iqr_outlier_share(train[c].values) for c in ["age","hours_per_week","capital_gain","capital_loss"]}
    print("Proporción de outliers (IQR) en train:")
    display(pd.Series(outlier_rates).sort_values(ascending=False).to_frame("share").round(4))

    # Boxplots por si quieres ilustrar en el reporte
    for c in ["age","hours_per_week","capital_gain","capital_loss"]:
        sns.boxplot(data=train, x="income", y=c)
        plt.title(f"Boxplot {c} por income")
        plt.show()

    # =========================
    # 8) Duplicados y consistencia
    # =========================
    dup_train = train.duplicated().mean()
    dup_test  = test.duplicated().mean()
    print(f"Duplicados - train: {dup_train:.4f} | test: {dup_test:.4f}")

    # Chequeo de consistencia target (solo para estar seguros)
    assert set(train["income"].unique()) <= {"<=50K",">50K"}
    assert set(test["income"].unique())  <= {"<=50K",">50K"}
else:
    print("File upload cancelled. Please upload the file to proceed.")

TypeError: 'NoneType' object is not subscriptable